In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
df1=pd.read_csv('../input/the-movies-dataset/credits.csv')
df2=pd.read_csv('../input/the-movies-dataset/movies_metadata.csv')
df4=pd.read_csv('../input/the-movies-dataset/keywords.csv')

In [0]:
df2 = df2[df2.id!='1997-08-20']
df2 = df2[df2.id!='2012-09-29']
df2 = df2[df2.id!='2014-01-01']

In [0]:
df2['id'] = df2['id'].astype(int)

In [0]:
df2=df2.merge(df1, on='id')

In [0]:
tfidf = TfidfVectorizer(stop_words='english')

In [0]:
df2['overview'] = df2['overview'].fillna('')

In [0]:
ran = random.randint(25000, 30000)
df3 = df2.head(ran)
tfidf_matrix = tfidf.fit_transform(df3['overview'])

In [0]:
cosine_sim =  linear_kernel(tfidf_matrix, tfidf_matrix, True)

In [0]:
indices = pd.Series(df3.index, index=df3['title']).drop_duplicates()

In [0]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df3['title'].iloc[movie_indices]

In [0]:
df2 = df2.merge(df4, on='id')

In [0]:
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(literal_eval)

In [0]:
#function to return director name from crew column
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [0]:
#function to get names of genres, cast and keywords from the data
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [0]:
df2['director'] = df2['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [0]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [0]:
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)

In [0]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)

In [0]:
count = CountVectorizer(stop_words='english')

 
df5 = df2['soup'].head(20000)

count_matrix = count.fit_transform(df5)

In [0]:
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [0]:
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [ ]:
mv=input("Enter the movie: ")
get_recommendations(mv, cosine_sim2)